In [22]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [23]:
CLIENT_ID = '2OOVLGGLIS3N5LE33BHXBFDRIBQ3N3YBMZMG3Y52TGNSJH4P' # your Foursquare ID
CLIENT_SECRET = 'CXE5SMYQEE5VINHNS4CXCUHRRUEF4AEVY4OQTBU0JMH24JYT' # your Foursquare Secret
VERSION = '20180618'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2OOVLGGLIS3N5LE33BHXBFDRIBQ3N3YBMZMG3Y52TGNSJH4P
CLIENT_SECRET:CXE5SMYQEE5VINHNS4CXCUHRRUEF4AEVY4OQTBU0JMH24JYT


In [24]:
address = '7000 Coliseum Way, Oakland, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.75027335 -122.202932326398


In [25]:
search_query = 'Playground'
radius = 50000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

Playground .... OK!


In [26]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.describe()

,location.distance,location.lat,location.lng
count,30.000000,30.000000,30.000000
mean,19946.266667,37.758068,-122.393561
std,4846.025773,0.073144,0.099571
min,3816.000000,37.548109,-122.502971
25%,19336.000000,37.739128,-122.449196
50%,20774.500000,37.763387,-122.430123
75%,22708.250000,37.787805,-122.389092
max,26472.000000,37.918952,-122.041954


In [27]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Mission Playground & Pool,Playground,1 Linda St,US,San Francisco,United States,NaN,19331,"[1 Linda St, San Francisco, CA 94110, United States]","[{'label': 'display', 'lat': 37.75943052670503, 'lng': -122.42226635988486}]",37.759431,-122.422266,94110,CA,4a84f3bbf964a520f1fd1fe3
1,Jackson Park & Playground,Park,1501 17th Street,US,San Francisco,United States,btw Arkansas & Carolina,17376,"[1501 17th Street (btw Arkansas & Carolina), San Francisco, CA 94107, United States]","[{'label': 'display', 'lat': 37.76430297307453, 'lng': -122.39957213401794}]",37.764303,-122.399572,94107,CA,4acab409f964a520dac220e3
2,Raymond Kimbell Playground,Playground,1901 Geary Blvd,US,San Francisco,United States,Steiner St,20740,"[1901 Geary Blvd (Steiner St), San Francisco, CA 94115, United States]","[{'label': 'display', 'lat': 37.78319767097322, 'lng': -122.43491012880098}]",37.783198,-122.434910,94115,CA,4ca2a4db554b236ac2402048
3,Playground,Karaoke Bar,1705 Buchanan St,US,San Francisco,United States,at Post St,20362,"[1705 Buchanan St (at Post St), San Francisco, CA 94115, United States]","[{'label': 'display', 'lat': 37.785764848386215, 'lng': -122.4299391113673}]",37.785765,-122.429939,94115,CA,45d5f376f964a520cf421fe3
4,Joe DiMaggio Playground,Playground,651 Lombard St,US,San Francisco,United States,at Powell,19351,"[651 Lombard St (at Powell), San Francisco, CA 94133, United States]","[{'label': 'display', 'lat': 37.802450196875085, 'lng': -122.41272760643703}]",37.802450,-122.412728,94133,CA,4af65b71f964a5203d0322e3
5,Walter Haas Park and Playground,Park,5499 Diamond Heights,US,San Francisco,United States,at Addison,20423,"[5499 Diamond Heights (at Addison), San Francisco, CA 94131, United States]","[{'label': 'display', 'lat': 37.7404989197731, 'lng': -122.43462258034437}]",37.740499,-122.434623,94131,CA,49f472ccf964a520076b1fe3
6,Julius Kahn Playground & Clubhouse,Playground,3206 W Pacific Ave,US,San Francisco,United States,at Spruce St,22514,"[3206 W Pacific Ave (at Spruce St), San Francisco, CA 94118, United States]","[{'label': 'display', 'lat': 37.79116336775309, 'lng': -122.45351135730743}]",37.791163,-122.453511,94118,CA,49f622b9f964a5200b6c1fe3
7,Moscone Park Playground,Playground,1890 Chestnut St,US,San Francisco,United States,NaN,21082,"[1890 Chestnut St, San Francisco, CA 94123, United States]","[{'label': 'display', 'lat': 37.801384703335295, 'lng': -122.43364816331051}]",37.801385,-122.433648,94123,CA,4b6a2fc9f964a5202acc2be3
8,Presidio Heights Playground,Playground,3437 Clay St,US,San Francisco,United States,Walnut St. & Laurel St.,22118,"[3437 Clay St (Walnut St. & Laurel St.), San Francisco, CA 94118, United States]","[{'label': 'display', 'lat': 37.78848466086141, 'lng': -122.44960458547621}]",37.788485,-122.449605,94118,CA,4c5c7313d25320a1c43bc37a
9,Golden Gate Park Mother's Playground,Playground,590 South Dr,US,San Francisco,United States,near Crossover,24120,"[590 South Dr (near Crossover), San Francisco, CA, United States]","[{'label': 'display', 'lat': 37.766848499143336, 'lng': -122.47620384137771}]",37.766848,-122.476204,NaN,CA,4af791bdf964a520e80922e3


In [32]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Oracle Arena',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the playgrounds as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [33]:
data = dataframe_filtered[['name','distance','lat','lng']]
data

,name,distance,lat,lng
0,Mission Playground & Pool,19331,37.759431,-122.422266
1,Jackson Park & Playground,17376,37.764303,-122.399572
2,Raymond Kimbell Playground,20740,37.783198,-122.434910
3,Playground,20362,37.785765,-122.429939
4,Joe DiMaggio Playground,19351,37.802450,-122.412728
5,Walter Haas Park and Playground,20423,37.740499,-122.434623
6,Julius Kahn Playground & Clubhouse,22514,37.791163,-122.453511
7,Moscone Park Playground,21082,37.801385,-122.433648
8,Presidio Heights Playground,22118,37.788485,-122.449605
9,Golden Gate Park Mother's Playground,24120,37.766848,-122.476204


In [35]:
X = data[['distance','lat','lng']]

In [43]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=1)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
y_kmeans

ImportError: cannot import name 'Kmeans'

In [41]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Oracle Arena',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the first kind playgrounds as blue circle markers
for lat, lng, label,y in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories, dataframe_filtered.y):
    if y == 0:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            popup=label,
            fill = True,
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

# add the second kind playgrounds as blue circle markers
for lat, lng, label,y in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories, dataframe_filtered.y):
    if y == 1:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            popup=label,
            fill = True,
            fill_color='green',
            fill_opacity=0.6
        ).add_to(venues_map)
        
# add the third kind playgrounds as blue circle markers
for lat, lng, label,y in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories, dataframe_filtered.y):
    if y == 0:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            popup=label,
            fill = True,
            fill_color='yellow',
            fill_opacity=0.6
        ).add_to(venues_map)
# display map
venues_map